# Install Packages

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
!pip3 install spacy
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import io
import re
from pprint import pprint

# Retrieve and Convert File to Data Frame

In [ ]:
directory = 'Desktop/SciFi_Texts/'

import pandas as pd
df = pd.read_csv('sci-fi_word_chunks_bag_of_words_output (1).csv')
df

In [ ]:
#Drop any empty cells
df = df.dropna()
df

In [ ]:
#Add values in Text column to new list (for further cleaning)
data = df.Text.values.tolist()

# Clean Texts

In [ ]:
#Define list of stopwords
stop_words = stopwords.words('english')

# Add further stopwords by simply "appending" desired words to dictionary
stop_words.append('CHAPTER')

In [ ]:
#Remove punctuation
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

In [ ]:
#Define function to perform simple preprocessing on text
def sent_to_words(sentences):
    for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [ ]:
#Run processing function on texts
data_words = list(sent_to_words(data))
#print(data_words[:1])

In [ ]:
#Define stopword removal
def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc))
if word not in stop_words] for doc in texts]

#Define function to make bigrams
def make_bigrams(texts):
   return [bigram_mod[doc] for doc in texts]

#def make_trigrams(texts):
#   return [trigram_mod[bigram_mod[doc]] for doc in texts]

#Define function to lemmatize texts
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
   texts_out = []
   for sent in texts:
     doc = nlp(" ".join(sent))
     texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
   return texts_out

In [ ]:
#Define bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
#Run functions to remove stopwords, make bigrams, and lemmatize text
data_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_nostops)
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:4])

# Building Dictionary and Corpus

In [ ]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus)

# Find Optimal # of Topics

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Get coherence value for topic models up to n = 100 topics
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words_nostops, start=2, limit=100, step=3) 

In [ ]:
# Show graph plotting coherence score for each topic model 
limit=100; start=2; step=3;
x = range(start, limit, step)
optimal, = plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
#Get coherence score for each num topics sorted from highest to lowest
#Highest value will be optimal number of topics
Data = {'Num Topics': optimal.get_xdata(), 'Coherence': optimal.get_ydata()}
df_optimal = pd.DataFrame(Data)
df_optimal.sort_values(by='Coherence', ascending=False)

# Create Topic Model with Optimal # Topics

In [ ]:
#Create topic model, inserting optimal number into num_topics
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14,
                                           random_state=100,
                                           update_every=2,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True) 


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_nostops, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
#Print n number of words in each topic
for idx, topic in lda_model.print_topics(num_words=8):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
# Create Visualization (Save HTML)
#The easiest way to create the visualization is to reveal it in the Google Colab notebook and save it as an html file that you can view on your browser. 

vis = gensimvis.prepare(lda_model, corpus, id2word)

In [ ]:
#Create and view visualization
vis = gensimvis.prepare(lda_model, corpus, id2word)

pyLDAvis.display(vis)

# Examine Topics Per Document

Find the topic with highest percentage in each document in corpus. 

Source: 
https://towardsdatascience.com/topic-modelling-in-python-with-spacy-and-gensim-dc8f7748bdbf

In [ ]:
#Define function that retrieves dominant topic for each document and puts in dataframe
def format_topics_texts(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    text_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                text_topics_df = text_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    text_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    text_topics_df = pd.concat([text_topics_df, contents], axis=1)
    return(text_topics_df)

In [ ]:
#Run dominant topic function on corpus
df_topic_texts_keywords = format_topics_texts(ldamodel=lda_model, corpus=corpus, texts=data_words_nostops)

# Format
df_dominant_topic = df_topic_texts_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

# Find Documents Per Topic
Calculate the total number of documents attributed to each topic in the model. 

Sources:

https://stackoverflow.com/questions/63777101/topic-wise-document-distribution-in-gensim-lda

https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/topic_methods.ipynb 

https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/#7.-The-most-representative-sentence-for-each-topic 

In [ ]:
#Add Book + Chapter labels to dataframe for easier ID
doc_names = df['Book + Chapter']
df_topic_texts_keywords = df_topic_texts_keywords.join(doc_names)

In [ ]:
# Get most representative text for each topic 
#Display setting to show more characters in column
pd.options.display.max_colwidth = 100

#Create new dataframe and group topic keywords by dominant topic column
topics_sorted_df = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_texts_keywords.groupby('Dominant_Topic')

#Sort data by percent contribution and select highest n values for each topic
for i, grp in sent_topics_outdf_grpd:
    topics_sorted_df = pd.concat([topics_sorted_df, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

In [ ]:
# Reset Index of new df
topics_sorted_df.reset_index(drop=True, inplace=True)

# Format
topics_sorted_df.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text", "Text Name"]
topics_sorted_df = topics_sorted_df.reindex(columns=['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text Name", "Representative Text"])
# Show
topics_sorted_df.head()

# Word Embedding with Gensim
Sources:

https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92 

https://www.shanelynn.ie/word-embeddings-in-python-with-spacy-and-gensim/

In [ ]:
#Get likelihood that a term belongs to certain topics
lda_model.get_term_topics('said')

In [ ]:
#Run word2vec on cleaned data
from gensim.models import Word2Vec
model = Word2Vec(data_words_nostops, min_count=1,size= 50,workers=3, window =3, sg = 1)

In [ ]:
model['tree']

In [ ]:
#compare similarity between two words
print(model.similarity('tree', 'space'))

print(model.similarity('space', 'sky'))

In [ ]:
#Get most similar words
model.most_similar('ocean')[:5]